In [5]:
import stratosphere_client
import json

stratosphere_base_url = "https://dev-stratosphere.blue-yonder.org"
token = "<YOUR-API-TOKEN>"
project_name = "fkeller-dask-dev"
service_name = "dask-demo"

def discover_endpoint(stratosphere_base_url, project_name, service_name, token):
    url = "{}/api/projects/{}".format(stratosphere_base_url, project_name)
    project = stratosphere_client.project.Project(token, url)
    service = project.get_service(service_name)

    status = json.loads(service.get_status().text)
    endpoint = status["endpoint"]

    return endpoint

endpoint = discover_endpoint(
    stratosphere_base_url, project_name, service_name, token
)
endpoint

u'tcp://d-platform-node9.unstable.ipc3.rack.zone:31968'

In [6]:
from dask.distributed import Client

client = Client(endpoint)

In [4]:
from dask.distributed import Client
# client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client = Client('scheduler:8786')
# client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

/venv/lib/python3.8/site-packages/distributed/client.py:1135: VersionMismatchWarning: Mismatched versions found

+---------+---------------+------------------------+------------------------+
| Package | client        | scheduler              | workers                |
+---------+---------------+------------------------+------------------------+
| blosc   | None          | 1.9.2                  | 1.9.2                  |
| dask    | 2021.02.0     | 2021.02.0+27.gadb58fa7 | 2021.02.0+27.gadb58fa7 |
| lz4     | None          | 3.0.2                  | 3.0.2                  |
| msgpack | 1.0.2         | 1.0.0                  | 1.0.0                  |
| numpy   | 1.20.1        | 1.19.4                 | 1.19.4                 |
| python  | 3.8.7.final.0 | 3.8.0.final.0          | 3.8.0.final.0          |
+---------+---------------+------------------------+------------------------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.Vers

Client Scheduler: tcp://scheduler:8786 Dashboard: http://scheduler:8787/status,Cluster Workers: 4 Cores: 4 Memory: 1.60 GB


In [5]:
import pandas as pd
import numpy as np
import time
import dask.dataframe as dd
from dask import delayed
from dask import visualize

def get_partitions(dataset_uuid):
    return [
        "{}_{}".format(dataset_uuid, i)
        for i in range(15)
    ]

def simulate_df_partition_load(partition):
    time.sleep(0.5)
    n = 1000000
    df = pd.DataFrame({
        "sales": np.random.poisson(lam=10, size=n),
        "prediction": np.random.poisson(lam=10, size=n),
        "product_id": np.random.choice([1, 2, 3], size=n),
        "location_id": np.random.choice([1, 2], size=n),
    })
    return df

dataset_uuid = "54e7a2b34"
partitions = get_partitions(dataset_uuid)

lazy_dataframes = [
    delayed(simulate_df_partition_load)(partition)
    for partition in partitions
]

df = dd.from_delayed(lazy_dataframes, meta=simulate_df_partition_load(0))

In [6]:
df = df.persist()

In [7]:
mad_dag = (df["prediction"] - df["sales"]).abs().mean()
mad_dag.visualize()

RuntimeError: Drawing dask graphs requires the `graphviz` python library and the `graphviz` system library to be installed.

In [8]:
mad = mad_dag.compute()
mad

KeyboardInterrupt: 